In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import pickle
from tqdm import tqdm

In [2]:
# Загружаем модель активности
lgbm_model = lgb.Booster(model_file='lgbm_activity_model.txt')

# Загружаем справочник с признаками пользователей
with open('users_with_features.pkl', 'rb') as f:
    users_with_features = pickle.load(f)
users_with_features.set_index('user_id', inplace=True)

# Загружаем модель аукциона (словарь с CPM)
with open('cpm_slot_lookup.pkl', 'rb') as f:
    cpm_slot_lookup = pickle.load(f)

# Список всех признаков, которые использовались при обучении модели
features_to_use = lgbm_model.feature_name()

In [8]:
features_to_use

['hour_of_day',
 'day_of_week',
 'sex',
 'age',
 'city_id',
 'total_impressions',
 'active_hours',
 'active_days',
 'unq_publishers',
 'cpm_mean',
 'cpm_median',
 'cpm_max',
 'avg_daily_impressions',
 'impressions_h_0',
 'impressions_h_1',
 'impressions_h_2',
 'impressions_h_3',
 'impressions_h_4',
 'impressions_h_5',
 'impressions_h_6',
 'impressions_h_7',
 'impressions_h_8',
 'impressions_h_9',
 'impressions_h_10',
 'impressions_h_11',
 'impressions_h_12',
 'impressions_h_13',
 'impressions_h_14',
 'impressions_h_15',
 'impressions_h_16',
 'impressions_h_17',
 'impressions_h_18',
 'impressions_h_19',
 'impressions_h_20',
 'impressions_h_21',
 'impressions_h_22',
 'impressions_h_23',
 'impressions_d_0',
 'impressions_d_1',
 'impressions_d_2',
 'impressions_d_3',
 'impressions_d_4',
 'impressions_d_5',
 'impressions_d_6',
 'ratio_impressions_h_0',
 'ratio_impressions_h_1',
 'ratio_impressions_h_2',
 'ratio_impressions_h_3',
 'ratio_impressions_h_4',
 'ratio_impressions_h_5',
 'ratio_im

In [41]:
# Функция вероятности выигрыша
def get_win_probability(my_cpm, publisher, hour, cpm_lookup):
    day_of_week = (hour // 24) % 7
    hour_of_day = hour % 24
    slot_key = (day_of_week, hour_of_day, publisher)
    competitor_cpms = cpm_lookup.get(slot_key)

    if competitor_cpms is None or len(competitor_cpms) == 0:
        return 0.03

    total_competitors = len(competitor_cpms)
    p_less = np.sum(my_cpm > competitor_cpms) / total_competitors
    p_equal = np.sum(my_cpm == competitor_cpms) / total_competitors
    win_prob = p_less + 0.5 * p_equal
    return win_prob

In [24]:
def predict_for_ad(ad_task, user_profiles, activity_model, auction_model):
    cpm = ad_task['cpm']
    hour_start = ad_task['hour_start']
    hour_end = ad_task['hour_end']
    publishers = ad_task['publishers'].split(',')
    user_ids = np.array(ad_task['user_ids'].split(',')).astype(int)
    audience_size = len(user_ids)

    if audience_size == 0:
        return {'at_least_one': 0.0, 'at_least_two': 0.0, 'at_least_three': 0.0}

    # Массив для хранения итогового числа просмотров для каждого пользователя
    total_views_per_user = np.zeros(audience_size, dtype=np.int8)

    # Предподсчет вероятностей выигрыша для этой кампании
    win_prob_cache = {}
    for hour in range(hour_start, hour_end + 1):
        # Усредняем p_win по всем разрешенным площадкам для каждого часа
        avg_p_win_for_hour = np.mean([
            get_win_probability(cpm, pub, hour, auction_model) for pub in publishers
        ])
        win_prob_cache[hour] = avg_p_win_for_hour

    # Получаем профили всех нужных пользователей
    audience_profiles = user_profiles.loc[user_profiles.index.intersection(user_ids)]

    # Итерируемся по каждому пользователю из аудитории
    for i, user_id in enumerate(user_ids):

        last_view_hour = -100

        # Проверяем, есть ли профиль для этого пользователя
        if user_id not in audience_profiles.index:
            continue

        user_features = audience_profiles.loc[user_id]
        feature_values = user_features.to_dict()

        # Цикл по часам кампании
        for hour in range(hour_start, hour_end + 1):

            # Правило сессии (6 часов)
            if (hour - last_view_hour) < 6:
                continue

            # Предсказание активности
            day_of_week = (hour // 24) % 7
            hour_of_day = hour % 24

            # Собираем вектор признаков для модели
            feature_values['hour_of_day'] = hour_of_day
            feature_values['day_of_week'] = day_of_week

            # reindex гарантирует правильный порядок колонок
            model_input = pd.DataFrame([feature_values]).reindex(columns=features_to_use, fill_value=0)

            lambda_val = activity_model.predict(model_input)[0]
            if lambda_val < 0: lambda_val = 0

            # Генерируем случайное целое число возможностей
            N_opps = np.random.poisson(lam=lambda_val)

            if N_opps == 0:
                continue

            # Берем предподсчитанную среднюю вероятность выигрыша для этого часа
            avg_p_win = win_prob_cache[hour]

            # Симулируем N_opps независимых аукционов
            hourly_wins = np.random.binomial(n=N_opps, p=avg_p_win)

            if hourly_wins > 0:
                total_views_per_user[i] += 1
                last_view_hour = hour # Обновляем время показа

    # Агрегация результатов
    at_least_one = np.sum(total_views_per_user >= 1) / audience_size
    at_least_two = np.sum(total_views_per_user >= 2) / audience_size
    at_least_three = np.sum(total_views_per_user >= 3) / audience_size

    return {
        'at_least_one': at_least_one,
        'at_least_two': at_least_two,
        'at_least_three': at_least_three,
    }


In [39]:
def predict_for_ad_optimized(ad_task, user_profiles, activity_model, auction_model):
    cpm = ad_task['cpm']
    hour_start = ad_task['hour_start']
    hour_end = ad_task['hour_end']
    publishers = ad_task['publishers'].split(',')
    user_ids = np.array(ad_task['user_ids'].split(','), dtype=np.int64)
    audience_size = len(user_ids)

    if audience_size == 0:
        return {'at_least_one': 0.0, 'at_least_two': 0.0, 'at_least_three': 0.0}

    total_views_per_user = np.zeros(audience_size, dtype=np.int8)
    last_view_hours = np.full(audience_size, -100, dtype=np.int32)

    win_prob_cache = {}
    for hour in range(hour_start, hour_end + 1):
        win_prob_cache[hour] = np.mean([
            get_win_probability(cpm, pub, hour, auction_model) for pub in publishers
        ])

    audience_profiles = user_profiles.reindex(user_ids).fillna(0)

    print("Запуск оптимизированной симуляции...")
    for hour in range(hour_start, hour_end + 1):
        model_input = audience_profiles.copy()
        model_input['hour_of_day'] = hour % 24
        model_input['day_of_week'] = (hour // 24) % 7

        X_predict = model_input[features_to_use]

        lambda_vals = activity_model.predict(X_predict)
        lambda_vals[lambda_vals < 0] = 0

        N_opps_all_users = np.random.poisson(lam=lambda_vals)

        avg_p_win = win_prob_cache[hour]

        hourly_wins_all_users = np.random.binomial(n=N_opps_all_users, p=avg_p_win)

        is_in_cooldown = (hour - last_view_hours) < 6

        hourly_wins_all_users[is_in_cooldown] = 0

        won_this_hour_mask = hourly_wins_all_users > 0

        total_views_per_user[won_this_hour_mask] += 1
        last_view_hours[won_this_hour_mask] = hour

    at_least_one = np.sum(total_views_per_user >= 1) / audience_size
    at_least_two = np.sum(total_views_per_user >= 2) / audience_size
    at_least_three = np.sum(total_views_per_user >= 3) / audience_size

    return {
        'at_least_one': at_least_one,
        'at_least_two': at_least_two,
        'at_least_three': at_least_three,
    }

In [43]:
validate_df = pd.read_csv('validate.tsv', sep='\t')
ad_task_to_test = validate_df.iloc[3]

print(ad_task_to_test[['cpm', 'hour_start', 'hour_end', 'publishers', 'audience_size']])
print("-" * 40)

predictions = predict_for_ad_optimized(
    ad_task=ad_task_to_test,
    user_profiles=users_with_features,
    activity_model=lgbm_model,
    auction_model=cpm_slot_lookup
)

print(predictions)


cpm              240.0
hour_start        1295
hour_end          1377
publishers        1,14
audience_size      440
Name: 3, dtype: object
----------------------------------------
Запуск оптимизированной симуляции...
{'at_least_one': np.float64(0.275), 'at_least_two': np.float64(0.11136363636363636), 'at_least_three': np.float64(0.025)}


In [46]:
N_ROWS_TO_PROCESS = 100
validate_subset = validate_df.head(N_ROWS_TO_PROCESS)

all_predictions = []

print(f"Запуск симуляции для {N_ROWS_TO_PROCESS} объявлений...")

for index, ad_task in validate_subset.iterrows():
    if (index + 1) % 10 == 0:
        print(f"  Обработано {index + 1}/{N_ROWS_TO_PROCESS} объявлений...")

    prediction = predict_for_ad_optimized(
        ad_task=ad_task,
        user_profiles=users_with_features,
        activity_model=lgbm_model,
        auction_model=cpm_slot_lookup,
    )
    all_predictions.append(prediction)



results_df = pd.DataFrame(all_predictions)
output_filename = "results.tsv"
results_df.to_csv(output_filename, sep='\t', index=False)

Запуск симуляции для 100 объявлений...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
  Обработано 10/100 объявлений...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
  Обработано 20/100 объявлений...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
Запуск оптимизированной симуляции...
Запус

**По итогу получили метрику 116.28%** и неоптимальную скорость работы. *(Был запущен скрипт из условия задачи)*